In [ ]:
import csv
import psycopg2
from psycopg2 import sql
import chardet
import pandas as pd  # Importando o Pandas

def detect_file_encoding(csv_file):
    """Detecta a codificação de um arquivo CSV usando chardet."""
    with open(csv_file, 'rb') as rawdata:
        result = chardet.detect(rawdata.read(800000))  # Lê os primeiros 800KB
    return result['encoding']

def create_table_from_csv(csv_file, table_name, db_config):
    """Cria uma tabela no PostgreSQL a partir de um arquivo CSV e salva em Pandas."""
    # Detecta a codificação do arquivo CSV
    encoding = detect_file_encoding(csv_file)
    print(f"Codificação detectada: {encoding}")
    
    # Conecta ao PostgreSQL
    conn = None
    cur = None
    try:
        print("Conectando ao banco de dados PostgreSQL...")
        conn = psycopg2.connect(**db_config)
        cur = conn.cursor()

        # Lê o CSV em um DataFrame do Pandas e exibe a amostra
        df = pd.read_csv(csv_file, delimiter=';', encoding=encoding, on_bad_lines='skip')
        print("Amostra dos dados:\n", df.head())

        # Criação da tabela com tipos mapeados
        type_mapping = {
            'ChEMBL ID': 'TEXT',
            'Assay Type': 'TEXT',
            'Description': 'TEXT',
            'Organism': 'TEXT',
            'Compounds': 'INTEGER',
            'Activities': 'INTEGER',
            'BAO Format ID': 'TEXT',
            'BAO Format': 'TEXT',
            'Confidence Score': 'INTEGER',
            'Confidence Label': 'TEXT',
            'Strain': 'TEXT',
            'Source': 'TEXT',
            'Tax ID': 'INTEGER',
            'Tissue ChEMBL ID': 'TEXT',
            'Tissue Name': 'TEXT',
            'Cell Type': 'TEXT',
            'Subcellular Fraction': 'TEXT',
            'Document ChEMBL ID': 'TEXT',
            'PubMed ID': 'TEXT',
            'DOI': 'TEXT',
            'Year': 'INTEGER',
            'Journal': 'TEXT',
            'Volume': 'TEXT',
            'First Page': 'TEXT',
            'Last Page': 'TEXT',
            'Assay Parameters': 'TEXT',
            'Assay Classification L1': 'TEXT',
            'Assay Classification L2': 'TEXT',
            'Assay Classification L3': 'TEXT',
            'Variant Sequence Accession': 'TEXT',
            'Variant Sequence Mutation': 'TEXT'
        }

        # Gera comando SQL para criar a tabela
        create_table_query = sql.SQL("CREATE TABLE IF NOT EXISTS {} (\n").format(sql.Identifier(table_name))
        columns = [
            sql.SQL("{} {}").format(sql.Identifier(col), sql.SQL(type_mapping.get(col, 'TEXT')))
            for col in df.columns
        ]
        create_table_query += sql.SQL(",\n").join(columns)
        create_table_query += sql.SQL("\n);")

        # Executa comando para criar a tabela
        print("Executando comando SQL para criar a tabela...")
        cur.execute(create_table_query)
        conn.commit()
        print(f"Tabela {table_name} criada com sucesso!")

        # Inserção dos dados
        insert_query = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(table_name),
            sql.SQL(', ').join(map(sql.Identifier, df.columns)),
            sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
        )

        # Substitui strings vazias por None para colunas numéricas e insere os dados
        for _, row in df.iterrows():
            data = [
                None if pd.isna(cell) else cell for cell in row
            ]
            cur.execute(insert_query, data)

        conn.commit()
        print(f"Dados inseridos com sucesso na tabela {table_name}!")

        # Salva os dados em uma planilha Excel
        output_file = f"{table_name}.xlsx"
        df.to_excel(output_file, index=False)
        print(f"Dados salvos em {output_file}!")

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Erro: {error}")
        if conn is not None:
            conn.rollback()
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()
        print("Conexão com o banco de dados fechada.")




        # Configuração do banco de dados
db_config = {
    'dbname': 'dtn',
    'user': 'postgres',
    'password': 'password',
    'host': 'localhost',
    'port': 54321
        }

        # Nome do arquivo CSV e da tabela
csv_file = 'assays-leishmania-functional.csv'
table_name = 'assays_leishmaniose'

        # Cria a tabela e insere os dados
create_table_from_csv(csv_file, table_name, db_config)

Codificação detectada: ascii
Conectando ao banco de dados PostgreSQL...
Amostra dos dados:
        ChEMBL ID Assay Type  \
0  CHEMBL1021351          F   
1  CHEMBL1074111          F   
2  CHEMBL1070291          F   
3   CHEMBL707048          F   
4   CHEMBL703309          F   

                                         Description                 Organism  \
0  Antileishmanial activity against Leishmania do...      Leishmania donovani   
1  Ratio of suberoylanilide hydroxamic acid IC90 ...      Leishmania donovani   
2  Antileishmanial activity against Leishmania do...      Leishmania donovani   
3  Tested in vitro for leishmanicidal activity ag...  Leishmania braziliensis   
4  Antiparasitic activity against Leishmania dono...      Leishmania donovani   

   Compounds  Activities BAO Format ID             BAO Format  \
0         12          12   BAO_0000218  organism-based format   
1          6           6   BAO_0000218  organism-based format   
2          1           1   BAO_0000218 